In [1]:
import pickle
import random
import numpy
import torch
import os

from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Memory_Network.Memory_Net import MemoryNetwork
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
current_trial = 0

torch.manual_seed(current_trial)
numpy.random.seed(current_trial)
random.seed(current_trial)
torch.cuda.manual_seed(current_trial)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
with open('Auxiliary_Functions/sensing_matrices/80 x 80 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/80 x 80 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/80 x 80 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('Auxiliary_Functions/sensing_matrices/80 x 80 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [4]:
with open('ILSVRC2012_80x80_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [5]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [6]:
projector = DnCNN(depth = 10)

# Equal Weighting
loss_function = IntermediateScaledLoss(omega = 1.0)

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# 5 Projections
num_projections = 5

# Folder Path for 5 Projections
proj5_path = 'Gauss_80_Depth10/5 Projections'

gauss_10_proj5_weights, gauss_10_proj5_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj5', display=True)

gauss_20_proj5_weights, gauss_20_proj5_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj5', display=True)

gauss_30_proj5_weights, gauss_30_proj5_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj5', display=True)

gauss_40_proj5_weights, gauss_40_proj5_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj5', display=True)

# Store Best Model Weights and History

save_progress(proj5_path, gauss_10_proj5_weights, gauss_10_proj5_hist, '10% Sampling')
save_progress(proj5_path, gauss_20_proj5_weights, gauss_20_proj5_hist, '20% Sampling')
save_progress(proj5_path, gauss_30_proj5_weights, gauss_30_proj5_hist, '30% Sampling')
save_progress(proj5_path, gauss_40_proj5_weights, gauss_40_proj5_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 0.2661012709273232
Average test loss: 0.015133730047278934
Epoch 2/300
Average training loss: 0.10175331284602483
Average test loss: 0.010980744846993023
Epoch 3/300
Average training loss: 0.08266142306725184
Average test loss: 0.010729624756508403
Epoch 4/300
Average training loss: 0.07306483250194126
Average test loss: 0.009854980358646976
Epoch 5/300
Average training loss: 0.06701451953914431
Average test loss: 0.008196130222744413
Epoch 6/300
Average training loss: 0.061363390054967666
Average test loss: 0.007948949226074748
Epoch 7/300
Average training loss: 0.058899383114443885
Average test loss: 0.00768581856538852
Epoch 8/300
Average training loss: 0.056095455507437386
Average test loss: 0.008352691486063931
Epoch 9/300
Average training loss: 0.05437393066949314
Average test loss: 0.008924828054176437
Epoch 10/300
Average training loss: 0.05167401100198428
Average test loss: 0.008766891626848116
Epoch 1

In [8]:
gauss_10_proj5_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj5_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj5_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj5_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj5_psnr = average_PSNR(gauss_10_proj5_model, gauss_10_proj5_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj5_psnr = average_PSNR(gauss_20_proj5_model, gauss_20_proj5_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj5_psnr = average_PSNR(gauss_30_proj5_model, gauss_30_proj5_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj5_psnr = average_PSNR(gauss_40_proj5_model, gauss_40_proj5_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj5_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj5_psnr, f)
with open(os.path.join(proj5_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj5_psnr, f)
with open(os.path.join(proj5_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj5_psnr, f)
with open(os.path.join(proj5_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj5_psnr, f)

/home/ubuntu/Memory-Net-Inverse/Denoising_Algorithms/DL_Training/evaluation.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(directory, 

Average PSNR for Projection Layer 0 across 2500 images: 22.45
Average PSNR for Projection Layer 1 across 2500 images: 24.69
Average PSNR for Projection Layer 2 across 2500 images: 26.01
Average PSNR for Projection Layer 3 across 2500 images: 26.65
Average PSNR for Projection Layer 4 across 2500 images: 27.28
Average PSNR for Projection Layer 0 across 2500 images: 23.63
Average PSNR for Projection Layer 1 across 2500 images: 26.84
Average PSNR for Projection Layer 2 across 2500 images: 28.53
Average PSNR for Projection Layer 3 across 2500 images: 29.55
Average PSNR for Projection Layer 4 across 2500 images: 30.21
Average PSNR for Projection Layer 0 across 2500 images: 24.48
Average PSNR for Projection Layer 1 across 2500 images: 28.08
Average PSNR for Projection Layer 2 across 2500 images: 30.01
Average PSNR for Projection Layer 3 across 2500 images: 31.24
Average PSNR for Projection Layer 4 across 2500 images: 32.07
Average PSNR for Projection Layer 0 across 2500 images: 24.88
Average 

In [ ]:
# 15 Projections
num_projections = 15

# Folder Path for 15 Projections
proj15_path = 'Gauss_80_Depth10/15 Projections'

gauss_10_proj15_weights, gauss_10_proj15_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj15', display=True)

gauss_20_proj15_weights, gauss_20_proj15_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj15', display=True)

gauss_30_proj15_weights, gauss_30_proj15_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj15', display=True)

gauss_40_proj15_weights, gauss_40_proj15_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj15', display=True)

# Store Best Model Weights and History

save_progress(proj15_path, gauss_10_proj15_weights, gauss_10_proj15_hist, '10% Sampling')
save_progress(proj15_path, gauss_20_proj15_weights, gauss_20_proj15_hist, '20% Sampling')
save_progress(proj15_path, gauss_30_proj15_weights, gauss_30_proj15_hist, '30% Sampling')
save_progress(proj15_path, gauss_40_proj15_weights, gauss_40_proj15_hist, '40% Sampling')

Validation Size: 4500, Train Size: 18000
Epoch 1/300
Average training loss: 5.166295516755846
Average test loss: 0.19637624008788002
Epoch 2/300
Average training loss: 2.3697386662165325
Average test loss: 0.013220546376374032
Epoch 3/300
Average training loss: 1.6505255421532525
Average test loss: 0.015091923822131422
Epoch 4/300
Average training loss: 1.289055707189772
Average test loss: 0.5335577875441975
Epoch 5/300
Average training loss: 1.0221601295471192
Average test loss: 0.08858109874195523
Epoch 6/300
Average training loss: 0.8518670448727078
Average test loss: 0.009316203507284323
Epoch 7/300
Average training loss: 0.7269524471494887
Average test loss: 0.0075367666565709646
Epoch 8/300
Average training loss: 0.6222062574492561
Average test loss: 0.007689244284811947
Epoch 9/300
Average training loss: 0.4701465707090166
Average test loss: 0.006763435216413604
Epoch 11/300
Average training loss: 0.41442795753479006
Average test loss: 0.009789914715621207
Epoch 12/300
Average t

In [ ]:
gauss_10_proj15_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj15_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj15_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj15_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj15_psnr = average_PSNR(gauss_10_proj15_model, gauss_10_proj15_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj15_psnr = average_PSNR(gauss_20_proj15_model, gauss_20_proj15_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj15_psnr = average_PSNR(gauss_30_proj15_model, gauss_30_proj15_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj15_psnr = average_PSNR(gauss_40_proj15_model, gauss_40_proj15_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj15_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj15_psnr, f)
with open(os.path.join(proj15_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj15_psnr, f)
with open(os.path.join(proj15_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj15_psnr, f)
with open(os.path.join(proj15_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj15_psnr, f)

In [ ]:
# 30 Projections
num_projections = 30

# Folder Path for 30 Projections
proj30_path = 'Gauss_80_Depth10/30 Projections'

gauss_10_proj30_weights, gauss_10_proj30_hist = train_main(gaussian_10_normalized, [1] * num_projections, gaussian_10_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_10_proj30', display=True)

gauss_20_proj30_weights, gauss_20_proj30_hist = train_main(gaussian_20_normalized, [1] * num_projections, gaussian_20_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_20_proj30', display=True)

gauss_30_proj30_weights, gauss_30_proj30_hist = train_main(gaussian_30_normalized, [1] * num_projections, gaussian_30_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_30_proj30', display=True)

gauss_40_proj30_weights, gauss_40_proj30_hist = train_main(gaussian_40_normalized, [1] * num_projections, gaussian_40_train, model=projector, device=device, model_type='Memory', split=0.2, 
                                                     numProjections=num_projections, residual=True, loss_function=loss_function, model_dir='Model_Paths/Gaussian_40_proj30', display=True)

# Store Best Model Weights and History

save_progress(proj30_path, gauss_10_proj30_weights, gauss_10_proj30_hist, '10% Sampling')
save_progress(proj30_path, gauss_20_proj30_weights, gauss_20_proj30_hist, '20% Sampling')
save_progress(proj30_path, gauss_30_proj30_weights, gauss_30_proj30_hist, '30% Sampling')
save_progress(proj30_path, gauss_40_proj30_weights, gauss_40_proj30_hist, '40% Sampling')

In [ ]:
gauss_10_proj30_model = MemoryNetwork(gaussian_10_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_20_proj30_model = MemoryNetwork(gaussian_20_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_30_proj30_model = MemoryNetwork(gaussian_30_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)
gauss_40_proj30_model = MemoryNetwork(gaussian_40_normalized, [1] * num_projections, num_projections, projector, device, residual=True).to(device)

gauss_10_proj30_psnr = average_PSNR(gauss_10_proj30_model, gauss_10_proj30_weights, num_projections, gaussian_10_test, print_psnr=True)
gauss_20_proj30_psnr = average_PSNR(gauss_20_proj30_model, gauss_20_proj30_weights, num_projections, gaussian_20_test, print_psnr=True)
gauss_30_proj30_psnr = average_PSNR(gauss_30_proj30_model, gauss_30_proj30_weights, num_projections, gaussian_30_test, print_psnr=True)
gauss_40_proj30_psnr = average_PSNR(gauss_40_proj30_model, gauss_40_proj30_weights, num_projections, gaussian_40_test, print_psnr=True)

with open(os.path.join(proj30_path, '10% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_10_proj30_psnr, f)
with open(os.path.join(proj30_path, '20% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_20_proj30_psnr, f)
with open(os.path.join(proj30_path, '30% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_30_proj30_psnr, f)
with open(os.path.join(proj30_path, '40% Sampling', 'test_PSNR.pkl'), 'wb') as f:
    pickle.dump(gauss_40_proj30_psnr, f)